In [1]:
import os 

In [2]:
%pwd

'e:\\End-to-end-text-summarization\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'e:\\End-to-end-text-summarization'

create entity 

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    data_path: Path
    model_path: Path
    tokenizer_path: Path
    metric_file_name: Path

In [6]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_file_path = CONFIG_FILE_PATH,
        params_file_path = PARAMS_FILE_PATH,
    ):
        
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
        
        create_directories([self.config.artifacts_root])
        
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation 
        
        create_directories([config.root_dir])
        
        model_evaluation_config = ModelEvaluationConfig(
            root_dir = config.root_dir,
            data_path = config.data_path,
            model_path = config.model_path,
            tokenizer_path= config.tokenizer_path,
            metric_file_name= config.metric_file_name,
        )
        return model_evaluation_config

create components

In [10]:
!pip install evaluate

In [11]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import load_dataset,  load_from_disk
import torch
import pandas as pd
from tqdm import tqdm
import evaluate

In [12]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config
        
    def generate_batch_sized_chunks(list_of_elements, batch_size):
        """split the dataset into smaller batches that we can process simultaneously
        Yield successive batch-sized chunks from list_of_elements."""
        for i in range(0, len(list_of_elements), batch_size):
            yield list_of_elements[i : i + batch_size]



    def calculate_metric_on_test_ds(dataset, metric, model, tokenizer,
                                batch_size=16, device="cuda" if torch.cuda.is_available() else "cpu",
                                column_text="dialogue",
                                column_summary="summary"):
    
        article_batches = list(generate_batch_sized_chunks(dataset[column_text], batch_size))
        target_batches = list(generate_batch_sized_chunks(dataset[column_summary], batch_size))

        all_predictions = []
        all_references = []

        for article_batch, target_batch in tqdm(
            zip(article_batches, target_batches), total=len(article_batches)):

            inputs = tokenizer(article_batch, max_length=1024,  truncation=True,
                            padding="max_length", return_tensors="pt")

            summaries = model.generate(input_ids=inputs["input_ids"].to(device),
                            attention_mask=inputs["attention_mask"].to(device),
                            length_penalty=0.8, num_beams=8, max_length=128)

            decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True,
                                    clean_up_tokenization_spaces=True)
                for s in summaries]

            decoded_summaries = [d.replace("", " ") for d in decoded_summaries]

            all_predictions.extend(decoded_summaries)
            all_references.extend(target_batch)

        #  Use compute directly with lists of predictions and references
        score = metric.compute(predictions=all_predictions, references=all_references)
        return score

    
    def evaluate(self):
        device = "cuda" if torch.cuda.is_available() else "cpu"
        tokenizer = AutoTokenizer.from_pretrained(self.config.tokenizer_path)
        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_path).to(device)
        
        # Loading data
        dataset_samsum_pt = load_from_disk(self.config.data_path)
        
        
        rouges_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]
        
        rouge_metric = evaluate.load('rouge')
        
        score = self.calculate_metric_on_test_ds(
            dataset_samsum_pt['test'][0:10], rouge_metric, model_pegasus, tokenizer, batch_size = 2, column_text= 'dialogue', column_summary='summary')
        
        rouge_dict = dict((rn, score[rn]) for rn in rouges_names)  # 


        df = pd.DataFrame(rouge_dict, index = ['pegasus'] )
        df.to_csv(self.config.metric_file_name, index=False)

In [14]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    def generate_batch_sized_chunks(self, list_of_elements, batch_size):
        """Split the dataset into smaller batches that we can process simultaneously.
        Yield successive batch-sized chunks from list_of_elements."""
        for i in range(0, len(list_of_elements), batch_size):
            yield list_of_elements[i: i + batch_size]

    def calculate_metric_on_test_ds(self, dataset, metric, model, tokenizer,
                                    batch_size=16, device="cuda" if torch.cuda.is_available() else "cpu",
                                    column_text="dialogue",
                                    column_summary="summary"):
        article_batches = list(self.generate_batch_sized_chunks(dataset[column_text], batch_size))
        target_batches = list(self.generate_batch_sized_chunks(dataset[column_summary], batch_size))

        all_predictions = []
        all_references = []

        for article_batch, target_batch in tqdm(
            zip(article_batches, target_batches), total=len(article_batches)):

            inputs = tokenizer(article_batch, max_length=1024, truncation=True,
                               padding="max_length", return_tensors="pt")

            summaries = model.generate(input_ids=inputs["input_ids"].to(device),
                                       attention_mask=inputs["attention_mask"].to(device),
                                       length_penalty=0.8, num_beams=8, max_length=128)

            decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True,
                                                  clean_up_tokenization_spaces=True)
                                 for s in summaries]

            decoded_summaries = [d.replace("", " ") for d in decoded_summaries]

            all_predictions.extend(decoded_summaries)
            all_references.extend(target_batch)

        # Compute the final ROUGE scores
        score = metric.compute(predictions=all_predictions, references=all_references)
        return score

    def evaluate(self):
        device = "cuda" if torch.cuda.is_available() else "cpu"
        tokenizer = AutoTokenizer.from_pretrained(self.config.tokenizer_path)
        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_path).to(device)

        # Load dataset
        dataset_samsum_pt = load_from_disk(self.config.data_path)

        rouges_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]
        rouge_metric = evaluate.load('rouge')

        score = self.calculate_metric_on_test_ds(
            dataset=dataset_samsum_pt['test'][0:10],
            metric=rouge_metric,
            model=model_pegasus,
            tokenizer=tokenizer,
            batch_size=2,
            column_text='dialogue',
            column_summary='summary'
        )

        rouge_dict = dict((rn, score[rn]) for rn in rouges_names)
        df = pd.DataFrame(rouge_dict, index=['pegasus'])
        df.to_csv(self.config.metric_file_name, index=False)


In [15]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.evaluate()
except Exception as e:
    raise e

[2025-06-01 19:21:06,297: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-06-01 19:21:06,299: INFO: common: yaml file: params.yaml loaded successfully]
[2025-06-01 19:21:06,300: INFO: common: Directory created: artifacts]
[2025-06-01 19:21:06,302: INFO: common: Directory created: artifacts/model_evaluation]


100%|██████████| 5/5 [03:50<00:00, 46.12s/it]

[2025-06-01 19:25:00,405: INFO: rouge_scorer: Using default tokenizer.]
